##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Estimadores pré-criados

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/premade"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/estimator/premade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

> Aviso: os Estimadores não são recomendados para novos códigos.  Os Estimadores executam código `v1.Session`, que é mais difícil de escrever corretamente e pode se comportar de forma inesperada, ainda mais quando usado em conjunto com código do TF 2. Os Estimadores são abarcados pelas [garantias de compatibilidade](https://tensorflow.org/guide/versions), mas não recebem mais correções, exceto para vulnerabilidades de segurança. Confira mais detalhes no [guia de migração](https://tensorflow.org/guide/migrate).

Este tutorial mostra como resolver o problema de classificação da íris no TensorFlow usando Estimadores. Um Estimador é uma representação de alto nível de um modelo completo legada do TensorFlow. Confira mais detalhes em [Estimadores](https://www.tensorflow.org/guide/estimator).

Observação: no TensorFlow 2.0, a [API Keras](https://www.tensorflow.org/guide/keras) pode realizar essas mesmas tarefas e é considerada uma API mais fácil de aprender. Se você estiver começando do zero, é recomendável usar o Keras.


## Em primeiro lugar

Para começar, você importará o TensorFlow e diversas bibliotecas necessárias.

In [ ]:
import tensorflow as tf

import pandas as pd

## O dataset

O programa de amostra neste documento cria e testa um modelo que classifica flores de íris em três espécies diferentes com base no tamanho das [sépalas](https://en.wikipedia.org/wiki/Sepal) e [pétalas](https://en.wikipedia.org/wiki/Petal).

Você treinará um modelo usando o dataset de íris. O dataset contém quatro características e um [rótulo](https://developers.google.com/machine-learning/glossary/#label). As quatro características identificam os seguintes aspectos botânicos das flores de íris individuais:

- comprimento da sépala
- largura da sépala
- comprimento da pétala
- largura da pétala

Com base nessas informações, você pode definir algumas constantes úteis para processar os dados:


In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

Em seguida, baixe e processe o dataset de íris usando o Keras e o Pandas. Mantenha datasets diferentes para treinamento e teste.

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

Você pode inspecionar seus dados para ver se tem quatro colunas de características float e um rótulo int32.

In [ ]:
train.head()

Para cada um dos datasets, divida os rótulos, que o modelo será treinado para prever.

In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

## Visão geral da programação com Estimadores

Agora que você configurou os dados, pode definir um modelo usando um Estimador do TensorFlow. Um Estimador é qualquer classe derivada do `tf.estimator.Estimator`. O TensorFlow oferece uma coleção de `tf.estimator` (por exemplo, `LinearRegressor`) para implementar algoritmos de ML comuns. Além disso, você pode escrever seus próprios [Estimadores personalizados](https://www.tensorflow.org/guide/estimator#custom_estimators). É recomendável usar Estimadores pré-criados quando estiver começando.

Para escrever um programa do TensorFlow baseado em Estimadores pré-criados, você precisa realizar as seguintes tarefas:

- Crie uma ou mais funções de entrada.
- Defina as colunas de características do modelo.
- Instancie um Estimador, especificando as colunas de características e vários hiperparâmetros.
- Chame um ou mais métodos no objeto Estimador, passando a função de entrada apropriada como a origem dos dados.

Vamos ver como essas tarefas são implementadas para a classificação de íris.

## Crie funções de entrada

Você precisa criar funções de entrada para fornecer dados para treinamento, avaliação e previsão.

Uma **função de entrada** retorna um objeto `tf.data.Dataset` que gera a seguinte tupla de dois elementos:

- [`features`](https://developers.google.com/machine-learning/glossary/#feature) — um dicionário no Python em que:
    - Cada chave é o nome de uma característica.
    - Cada valor é um array com todos os valores dessa característica.
- `label` — um array com os valores do [rótulo](https://developers.google.com/machine-learning/glossary/#label) para cada exemplo.

Só para demonstrar o formato da função de entrada, aqui está uma implementação simples:

In [ ]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

Sua função de entrada pode gerar o dicionário `features` e a lista `label` da maneira que você quiser. No entanto, é recomendável usar a [API Dataset](https://www.tensorflow.org/guide/datasets) do TensorFlow, que pode processar todos os tipos de dados.

A API Dataset consegue lidar com vários casos comuns para você. Por exemplo, usando a API Dataset, você pode ler facilmente os registros de uma grande coleção de arquivos em paralelo e combiná-los em um único fluxo.

Para simplificar as coisas, neste exemplo, você carregará os dados com o [pandas](https://pandas.pydata.org/) e criará um pipeline de entrada a partir destes dados na memória:


In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


## Defina as colunas de características

Uma [**coluna de características**](https://developers.google.com/machine-learning/glossary/#feature_columns) é um objeto que descreve como o modelo deve usar os dados brutos de entrada do dicionário "features". Ao criar um modelo de Estimador, você passa uma lista de colunas de características que descreve cada uma das características que você quer que o modelo use. O módulo `tf.feature_column` oferece várias opções de representação de dados para o modelo.

Para a íris, as 4 características brutas são valores numéricos, então você criará uma lista de colunas de características para dizer ao modelo do Estimador representar cada uma das quatro características como valores de pontos flutuantes de 32 bits. Portanto, o código para criar a coluna é o seguinte:

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

As colunas de características podem ser muito mais sofisticadas do que as mostradas aqui. Você pode ler mais sobre essas colunas [neste guia](https://www.tensorflow.org/guide/feature_columns).

Agora que você tem a descrição de como quer que o modelo represente as características brutas, pode criar o estimador.

## Instancie um estimador

A íris é um problema clássico de classificação. Felizmente, o TensorFlow oferece vários Estimadores classificadores pré-criados, incluindo:

- `tf.estimator.DNNClassifier` para modelos profundos que realizam a classificação de várias classes.
- `tf.estimator.DNNLinearCombinedClassifier` para modelos profundos e abrangentes.
- `tf.estimator.LinearClassifier` para classificadores com base em modelos lineares.

Para o problema da íris, `tf.estimator.DNNClassifier` parece ser a melhor opção. Veja como instanciar esse Estimador:

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

## Treine, avalie e faça previsões

Agora que você tem um objeto Estimador, é possível chamar os métodos para fazer o seguinte:

- Treinar o modelo.
- Avaliar o modelo treinado.
- Usar o modelo treinado para fazer previsões.

### Treine o modelo

Treine o modelo ao chamar o método `train` do Estimador da seguinte maneira:

In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Observe que a chamada de `input_fn` é empacotada em um [`lambda`](https://docs.python.org/3/tutorial/controlflow.html) para capturar os argumentos ao fornecer uma função de entrada que não aceita argumentos, conforme esperado pelo Estimador. O argumento `steps` diz ao método para interromper o treinamento após um determinado número de passos.


### Avalie o modelo treinado

Agora que o modelo foi treinado, você pode obter algumas estatísticas sobre o desempenho dele. O bloco de código a seguir avalia a exatidão do modelo treinado com os dados de teste:


In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

Ao contrário da chamada do método `train`, você não passou o argumento `steps` para avaliação. O `input_fn` para avaliação só gera uma única [época](https://developers.google.com/machine-learning/glossary/#epoch) de dados.

O dicionário `eval_result` também contém a `average_loss` (perda média por amostra), a `loss` (perda média por minilote) e o valor do `global_step` do Estimador (o número de iterações de treinamento realizadas).


### Fazer previsões (inferir) com o modelo treinado

Agora, você tem um modelo treinado que produz bons resultados de avaliação. É possível usar o modelo treinado para prever as espécies de uma flor de íris com base em algumas medidas não rotuladas. Assim como no treinamento e na avaliação, você faz previsões usando uma única chamada de função:

In [ ]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

O método `predict` retorna uma iterável em Python, gerando um dicionário de resultados previstos para cada exemplo. O código a seguir imprime algumas previsões e suas probabilidades.

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))